# Model for recomendations solutions on drainage.

1. Postarać się dodać tagi z klasami zlewni

## 1. Import libraries

In [1]:
import swmmio
import pyswmm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Normalization, Input
from tensorflow.keras.callbacks import EarlyStopping


# desired_width = 500
# pd.set_option("display.width", desired_width)
# np.set_printoptions(linewidth=desired_width)
# pd.set_option("display.max_columns", 30)
import pprint
pd.set_option('display.max_columns', None)
# Ustawienie maksymalnej ilości wyświetlanych wierszy
pd.set_option('display.max_rows', None)


## 2. Recomendation classes

In [2]:
classes = pd.DataFrame(
    data={
        "classes": [
            "valid",
            "diameter_increase",
            "diameter_reduction",
            "slope_increase",
            "slope_reduction",
            "depth_increase",
            "depth_reduction",
            "pump",
            "tank",
            "seepage_boxes",
        ]
    }
)
classes


,classes
0,valid
1,diameter_increase
2,diameter_reduction
3,slope_increase
4,slope_reduction
5,depth_increase
6,depth_reduction
7,pump
8,tank
9,seepage_boxes


## 3. Get data

In [3]:
import pandas as pd
import swmmio
import pyswmm
from sa.core.data import DataManager

diameter_01 = R"C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\recomendations\diameter\01_diameter.inp"
recomendations_01 = []


model = DataManager(in_file_path=diameter_01, include_rpt=True)
model.enter()
with pyswmm.Simulation(model.inp.path) as sim:
    for _ in sim:
        pass

print(model.df_conduits.columns)
print(model.df_conduits)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


TypeError: 'bool' object is not subscriptable

### 3.1. Select columns for training

In [ ]:
dataset = model.df_conduits[["ValMaxFill", "ValMaxV", "ValMinV", "ValMaxSlope", "ValMinSlope", "ValDepth", "ValCoverage", "isMinDiameter", "Tag"]]
# model.df_conduits.drop([
#     "MaxDay", "InletNode", "OutletNode", "MaxHr",
#      "InletNodeInvert",  "OutletNodeInvert",  "UpstreamInvert", "DownstreamInvert", "MaxFlow",
#      "InletMaxDepth", "OutletMaxDepth", "InletGroundElevation", "OutletGroundElevation", "InletGroundCover", "OutletGroundCover",
#                         ], inplace=True, axis=1)
print(dataset)
# categories = ["diameter_reduction", "valid", "depth_increase"]

NameError: name 'model' is not defined

### 3.2. Categories 

### 3.3. Classification conduits to categories

In [ ]:
dataset = dataset.reset_index(drop=True)
X = dataset.drop("Tag", axis=1)
y = dataset["Tag"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)
print(y_train, y_test)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

NameError: name 'dataset' is not defined

## 6. Build the model

In [ ]:
from keras.regularizers import l2

model = Sequential()
model.add(Dense(units=X_train.shape[1], activation='relu', input_dim=X_train.shape[1], kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1], activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=100,
    validation_data=(X_test, y_test),
    validation_split=0.2,
    verbose=1,
    batch_size=16,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10)]
)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6250 - loss: 0.8432 - val_accuracy: 0.5000 - val_loss: 0.8542
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.6250 - loss: 0.8424 - val_accuracy: 0.5000 - val_loss: 0.8541
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.6250 - loss: 0.8389 - val_accuracy: 0.5000 - val_loss: 0.8539
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.5000 - loss: 0.8414 - val_accuracy: 0.5000 - val_loss: 0.8537
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.6250 - loss: 0.8398 - val_accuracy: 0.5000 - val_loss: 0.8535
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.6250 - loss: 0.8291 - val_accuracy: 0.5000 - val_loss: 0.8534
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.6250 - loss: 0.8357 - val_accuracy: 0.5000 - val_loss: 0.8532
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.6250 - loss: 0.8283 - val_accuracy: 0.5000 - val_loss:

## 7. Evaluate model

In [ ]:
# Evaluate ANN model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test set accuracy: {accuracy:.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5000 - loss: 0.8486
Test set accuracy: 0.500


In [ ]:
# model.predict(X_test)
y_pred = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


In [ ]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

,accuracy,loss,val_accuracy,val_loss,epoch
0,0.625,0.843179,0.5,0.854182,0
1,0.625,0.842435,0.5,0.854064,1
2,0.625,0.838888,0.5,0.853940,2
3,0.500,0.841357,0.5,0.853721,3
4,0.625,0.839848,0.5,0.853509,4
5,0.625,0.829110,0.5,0.853357,5
6,0.625,0.835673,0.5,0.853228,6
7,0.625,0.828294,0.5,0.853159,7
8,0.625,0.823886,0.5,0.853156,8
9,0.500,0.849706,0.5,0.852991,9


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy', row=1, col=1)
fig.update_yaxes(title_text='loss', row=1, col=2)
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(test_acc)

0.5


## 8. Predictions

In [ ]:
predictions = model.predict(X_test)
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


array([[0.5504575 , 0.44954252],
       [0.58029795, 0.41970205]], dtype=float32)

In [ ]:
predictions_df = pd.DataFrame(predictions)
predictions_df

,0,1
0,0.550457,0.449543
1,0.580298,0.419702


In [ ]:
predictions_cls = predictions.argmax(axis=-1)
predictions_cls

array([0, 0], dtype=int64)

# Zapisanie modelu

In [ ]:
model.save(R'C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\recomendations\recomendations.keras')


In [ ]:
from tensorflow import keras
model = keras.models.load_model(R'C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\recomendations\recomendations.keras')
pred = model.predict(X_test)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


array([[0.5504575 , 0.44954252],
       [0.58029795, 0.41970205]], dtype=float32)